In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import tensorflow as tf
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 3000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'translation2019zh_train.json'
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
"""
with open('HW5.txt','w',encoding='utf-8') as out:
    
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()[:2500]
        for line in lines:
            l=line.split('"')
            target_text=l[7][:-1]
            input_text=l[3][:-1]
            out.write(input_text+"\t"+target_text+"\n")
"""
lines = open('HW5.txt', encoding ='utf-8').read().split('\n')
for line in lines[: 3000]:
    l= line.split('\t')
    input_text=l[0]
    target_text=l[1]
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print(target_texts)
print(input_texts)
  

['\t为了更好的锐度，但是附带的会多一些颗粒度，可以使用这个显影剂的1：1稀释液\n', '\t他还把宣扬自己思想的所谓《绿皮书》称作“新福音书”\n', '\t微风推着我去爱抚它的长耳\n', '\t它们的先烈们的鲜血是白流了\n', '\t最后，在1月31日，湖人将前往汽车城底特律挑战活塞队，活塞近来在东部排名第二\n', '\tchines\n', '\t2006年，沃尔玛的推荐引擎竟将《人猿星球》与马丁·路德·金的记录片配成了一对，为此沃尔玛遭到了种族歧视的指控\n', '\t通过电子探针显微分析确定贫化渣中主要铜相为冰铜相\n', '\t吉姆靠给人擦皮鞋为生\n', '\t用甘氨酸模拟胶原，研究间苯二酚-恶唑烷E鞣性基质的形成以及与胶原之间的反应特性\n', '\t免费市内取送、免费提供打印译稿及电子文档各一份\n', '\t基尔大学以其令人兴奋的方式高等教育，美丽的校园，强大的社区精神和优秀学生的生活\n', '\t这些中药中就有生长于云南和贵州的茯苓\n', '\t或者是因为你愿意花费无数的夜晚去安慰他们\n', '\tCallum：  OK，答案将会在节目的最后揭晓，到时我们再看你有没有答对\n', '\t当他站在水平的地面上时，后足爪位于身躯后方（不在身躯正下方），从脚垫到飞节垂直于地面\n', '\t所以到底是谁赢了？ 阿拉斯加不算数，那是你们从俄罗斯买的\n', '\t在一个特别寒冷的冬天，一对住在明尼阿波利斯市的夫妇决定去弗罗里达避寒\n', '\t热爱温暖的毛袜子和冰冻柠檬汁的邓布利多，变出柔软舒适的家具\n', '\t要成功拿到博士学位，你必须真正的扩展人类的知识边界\n', '\t这人说：\n', '\t逐步淘汰社会保障。社会保障并不起作用，而且它会使我们的国家破产\n', '\tchines\n', '\t看看这边的杯垫\n', '\t两队不分胜负，所以持续了很长一段时间\n', '\t本帖隐藏的内容需要回复才可以浏览2300作为一个独立的立体声编码器\n', '\t富勒是由美国和加拿大神学院联盟授权的253 家学院中的一员\n', '\t而腹板鼓曲对纯弯和纯剪极限承载力的影响则可不予考虑\n', '\tchines\n', '\t火星在本教程中获得击中从2001年太空漫游所有的一块完整的二维\n', '\t显示所有关于这个主题的文

In [2]:
# 以dict儲存字典單字及序號
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# 設定編碼器、解碼器input起始值(均為0矩陣)
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


# 設定 encoder_input、decoder_input對應的順序    
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [3]:
# 建立 encoder LSTM 隱藏層
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# 捨棄 output，只保留記憶狀態 h 及 c
encoder_states = [state_h, state_c]

In [4]:
# 建立 decoder LSTM 隱藏層
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# decoder 記憶狀態不會在訓練過程使用，只會在推論(Inference)使用
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 定義模型，由 encoder_input_data 及 decoder_input_data 轉換為 decoder_target_data 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 訓練
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
          
          
# 儲存模型及結果
model.save('HW5.h5')



Epoch 1/100
32/32 [==============================] - 112s 3s/step - loss: 1.9254 - val_loss: 1.8124
Epoch 2/100
32/32 [==============================] - 100s 3s/step - loss: 1.8659 - val_loss: 1.8025
Epoch 3/100
32/32 [==============================] - 99s 3s/step - loss: 1.8596 - val_loss: 1.7997
Epoch 4/100
32/32 [==============================] - 102s 3s/step - loss: 1.8569 - val_loss: 1.7982
Epoch 5/100
32/32 [==============================] - 103s 3s/step - loss: 1.8552 - val_loss: 1.7981
Epoch 6/100
32/32 [==============================] - 101s 3s/step - loss: 1.8538 - val_loss: 1.7957
Epoch 7/100
32/32 [==============================] - 103s 3s/step - loss: 1.8523 - val_loss: 1.7961
Epoch 8/100
32/32 [==============================] - 105s 3s/step - loss: 1.8521 - val_loss: 1.7981
Epoch 9/100
32/32 [==============================] - 103s 3s/step - loss: 1.8515 - val_loss: 1.7971
Epoch 10/100
32/32 [==============================] - 104s 3s/step - loss: 1.8510 - val_loss: 1.8006


KeyboardInterrupt: 

In [5]:
# 定義編碼器取樣模型
encoder_model = Model(encoder_inputs, encoder_states)
#
# 定義解碼器的input
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 定義解碼器 LSTM 模型
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
    
# 以編碼器的記憶狀態 h 及 c 為解碼器的記憶狀態  
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# 建立反向的 dict，才能透過查詢將數值轉回文字
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


# 模型預測，並取得翻譯結果(中文)    
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [6]:
"""
for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('*')
    print('Input sentence:', input_texts[seq_index])
    try:
        print('Decoded sentence:', decoded_sentence)
    except:
        # 出現亂碼，以?取代
        print('Decoded sentence:', decoded_sentence.encode('ascii', 'replace'))
        #print("error:", sys.exc_info()[0])
"""
valid_texts=list()
valid_characters = set()
with open('translation2019zh_valid.json','r',encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        l=line.split('"')
        valid_texts.append(l[3])
valid_text=[ valid_texts[i] for i in range(20)]        
for char in valid_text:
    if char not in valid_characters:
        valid_characters.add(char)
valid_characters = sorted(list(valid_characters))
valid_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
print(valid_token_index)
encoder_valid_data = np.zeros(
    (len(valid_text), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

for i, (l[3], l[7]) in enumerate(zip(l[3], l[7])):
    for t, char in enumerate(l[3]):
        encoder_valid_data[i, t, valid_token_index[char]] = 1.
for seq_index in range(10):
    input_seq = encoder_valid_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('*')
    print('Input sentence:', valid_text[seq_index])
    try:
        print('Decoded sentence:', decoded_sentence)
    except:
        # 出現亂碼，以?取代
        print('Decoded sentence:', decoded_sentence.encode('ascii', 'replace'))
        #print("error:", sys.exc_info()[0])

"""
m=tf.keras.models.load_model('HW5.h5')
prediction=m.predict('Slowly and not without struggle, America began to listen')
"""

{' ': 0, '!': 1, '$': 2, '%': 3, '&': 4, "'": 5, '(': 6, ')': 7, '*': 8, '+': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '<': 26, '=': 27, '>': 28, '?': 29, 'A': 30, 'B': 31, 'C': 32, 'D': 33, 'E': 34, 'F': 35, 'G': 36, 'H': 37, 'I': 38, 'J': 39, 'K': 40, 'L': 41, 'M': 42, 'N': 43, 'O': 44, 'P': 45, 'Q': 46, 'R': 47, 'S': 48, 'T': 49, 'U': 50, 'V': 51, 'W': 52, 'X': 53, 'Y': 54, 'Z': 55, '[': 56, '\\': 57, ']': 58, '_': 59, '`': 60, 'a': 61, 'b': 62, 'c': 63, 'd': 64, 'e': 65, 'f': 66, 'g': 67, 'h': 68, 'i': 69, 'j': 70, 'k': 71, 'l': 72, 'm': 73, 'n': 74, 'o': 75, 'p': 76, 'q': 77, 'r': 78, 's': 79, 't': 80, 'u': 81, 'v': 82, 'w': 83, 'x': 84, 'y': 85, 'z': 86, '~': 87, '´': 88, '·': 89, 'á': 90, 'â': 91, 'è': 92, 'ñ': 93, 'ö': 94, 'ü': 95, 'δ': 96, '–': 97, '—': 98, '‘': 99, '’': 100, '“': 101, '”': 102, '…': 103, '′': 104, '€': 105, '℃': 106, '™': 107, '─': 108, '。': 109, '上': 110

"\nm=tf.keras.models.load_model('HW5.h5')\nprediction=m.predict('Slowly and not without struggle, America began to listen')\n"